# Introdução

Este notebook apresenta a aplicação do método Gradient-weighted Class Activation Mapping (Grad-CAM) para interpretação das classificações realizadas por 5 CNNs distintas.

O Grad-CAM foi proposto por Selvaraju et al (2016) em:
"Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization" (disponível em https://arxiv.org/abs/1610.02391)

O presente trabalho utilizou a implementação do Grad-CAM apresentada em um exemplo da biblioteca Keras: https://keras.io/examples/vision/grad_cam/

# Configuração

Bibliotecas

In [ ]:
import os
import os.path as osp
import numpy as np
import pandas as pd
from shutil import copyfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers

Funções auxiliares

In [ ]:
# registro em log
def log(texto,name):

  with open('./'+name+'.txt',"a") as txt:
      txt.write(texto + '\n')

Funções de pré-processamento das imagens

In [ ]:
#VGG
def vgg_preprocess_input(x_temp):
    x_temp = x_temp[..., ::-1]
    x_temp[...,0] -= 93.5940
    x_temp[...,1] -= 104.7624
    x_temp[...,2] -= 129.1863
    
    return x_temp

#ResNet e SENet
def resnet_preprocess_input(x_temp):
    x_temp = x_temp[..., ::-1]
    x_temp[...,0] -= 91.4953
    x_temp[...,1] -= 103.8827
    x_temp[...,2] -= 131.0912
    
    return x_temp

#Inception-V3
incep_preprocess_input = tf.keras.applications.inception_v3.preprocess_input


#N-CNN
def ncnn_preprocess_input(x_temp):
    
    return x_temp/255


# Esta função seleciona o pré-processamento em função do modelo
def preprocess_input(x_temp, modelo):

  if modelo == 'VGG16':
    x_temp = vgg_preprocess_input(x_temp)

  elif modelo == 'ResNet50' or modelo == 'SENet50':
    x_temp = resnet_preprocess_input(x_temp)

  elif modelo == 'IncepV3':
    x_temp = incep_preprocess_input(x_temp)

  else:
    x_temp = ncnn_preprocess_input(x_temp)

  return x_temp


### Funções do Grad-CAM

Estas funções foram retiradas de um exemplo da biblioteca Keras, disponível em:
https://keras.io/examples/vision/grad_cam/

In [ ]:
from tensorflow import keras
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Esta função carrega uma imagem e a converte num arrat
def get_img_array(img_path, size):
    # `img` é uma PIL image de tamanho HxW
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` é float32 Numpy array de tamanho (H, W, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # Adiciona-se uma dimensão ao array para transformá-lo num "batch"
    # de dimensão (1, H, W, 3)
    array = np.expand_dims(array, axis=0)
    return array


# Esta função produz o Mapa de Ativação de Classe do Grad-CAM
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):

    # Inicialmente, é criada uma versão do modelo que mapeia a imagem de entrada
    # tanto na ativação da última cada convolucional e quanto na camada de saída.
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Em seguida, computa-se o gradiente da classe prevista para imagem de entrada
    # com relação a ativação da última camada convolucional
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        # canal de saída corresponde a classe determinada
        class_channel = preds[:, pred_index]  

    # Este é o gradiente do neurônio da saída em relação ao feature map gerado 
    # pela última camada convolucional
    grads = tape.gradient(class_channel, last_conv_layer_output)
    
    # Neste vetor, cada valor corresponde a média da intensidade do gradiente 
    # com relação a um canal específico do feature map.
    # Em outras palavras, é a operação de Global Average Pooling aplicada ao 
    # gradiente para produzir o "vetor de importâncias" dos canais
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Cada canal do feature map é multiplicado pelo valor correspondente no 
    # "vetor de importâncias". Em seguida, é feita a soma de todos os canais 
    # para a produção do mapa de ativação   
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # O mapa é normalizado entre 0 e 1 para possibilitar a visualização
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


# Esta função sobrepõe o Mapa de Ativação de Classe na imagem original
# e salva o resultado  
def save_gradcam(img_path, heatmap, img_size, cam_path="cam.jpg", alpha=0.4):
    # Carregando a imagem original
    img = keras.preprocessing.image.load_img(img_path, target_size=img_size)
    img = keras.preprocessing.image.img_to_array(img)

    # Ajuste do mapa para uma escala de 0-255
    heatmap = np.uint8(255 * heatmap)

    # Carregando o colormap 'jet'
    jet = cm.get_cmap("jet")

    # Aplicando o colormap no mapa
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Criando uma imagem RGB do mapa de ativacao colorido
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Sobreposicao do mapa de ativacao na imagem original
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Salvando o resultado da sobreposicao    
    superimposed_img.save(cam_path)

    # Apresentar o resultado da sobreposição na tela
    #display(Image(superimposed_img))




In [ ]:
!unzip -q ./COPE_UNIFESP_NBfolds.zip 
base_path = './COPE_UNIFESP_NBfolds'

!mkdir ./GradCAM_NBfold4

# Aplicação do Grad-CAM nos modelos

### Criação dos diretórios para salvar os resultados

In [ ]:
# VGG

# Resultados gerais
!mkdir ./VGG16_GradCAM
!mkdir ./VGG16_GradCAM/All
!mkdir ./VGG16_GradCAM/All/Right
!mkdir ./VGG16_GradCAM/All/Wrong

# Resultados separados por Fold da validação cruzada
for i in range(10):

  os.mkdir('./VGG16_GradCAM/Fold'+str(i))
  os.mkdir('./VGG16_GradCAM/Fold'+str(i)+'/Right')
  os.mkdir('./VGG16_GradCAM/Fold'+str(i)+'/Wrong')


In [ ]:
# ResNet50

# Resultados gerais
!mkdir ./ResNet50_GradCAM
!mkdir ./ResNet50_GradCAM/All
!mkdir ./ResNet50_GradCAM/All/Right
!mkdir ./ResNet50_GradCAM/All/Wrong

# Resultados separados por Fold da validação cruzada
for i in range(10):

  os.mkdir('./ResNet50_GradCAM/Fold'+str(i))
  os.mkdir('./ResNet50_GradCAM/Fold'+str(i)+'/Right')
  os.mkdir('./ResNet50_GradCAM/Fold'+str(i)+'/Wrong')


In [ ]:
# SENet50

# Resultados gerais
!mkdir ./SENet50_GradCAM
!mkdir ./SENet50_GradCAM/All
!mkdir ./SENet50_GradCAM/All/Right
!mkdir ./SENet50_GradCAM/All/Wrong

# Resultados separados por Fold da validação cruzada
for i in range(10):

  os.mkdir('./SENet50_GradCAM/Fold'+str(i))
  os.mkdir('./SENet50_GradCAM/Fold'+str(i)+'/Right')
  os.mkdir('./SENet50_GradCAM/Fold'+str(i)+'/Wrong')


In [ ]:
# Inception-V3

# Resultados gerais
!mkdir ./IncepV3_GradCAM
!mkdir ./IncepV3_GradCAM/All
!mkdir ./IncepV3_GradCAM/All/Right
!mkdir ./IncepV3_GradCAM/All/Wrong

# Resultados separados por Fold da validação cruzada
for i in range(10):

  os.mkdir('./IncepV3_GradCAM/Fold'+str(i))
  os.mkdir('./IncepV3_GradCAM/Fold'+str(i)+'/Right')
  os.mkdir('./IncepV3_GradCAM/Fold'+str(i)+'/Wrong')


In [ ]:
# N-CNN

# Resultados gerais
!mkdir ./NCNN_GradCAM
!mkdir ./NCNN_GradCAM/All
!mkdir ./NCNN_GradCAM/All/Right
!mkdir ./NCNN_GradCAM/All/Wrong

# Resultados separados por Fold da validação cruzada
for i in range(10):

  os.mkdir('./NCNN_GradCAM/Fold'+str(i))
  os.mkdir('./NCNN_GradCAM/Fold'+str(i)+'/Right')
  os.mkdir('./NCNN_GradCAM/Fold'+str(i)+'/Wrong')


##  Grad-CAM

Inicialmente, é preciso determinar qual modelo será submetido ao Grad-CAM.
Opções: VGG16, ResNet50, SENet50, Inception-V3 e N-CNN

In [ ]:
modelo = "VGG16"
#modelo = "ResNet50"
#modelo = "SENet50"
#modelo = "IncepV3"
#modelo = "NCNN"

Para a execução deste script, a base de dados (especificamente, a união das bases utilizadas) foi organizada em pastas que correspondem aos *folds* da validação cruzada, conforme o exemplo a seguir:

*   Base A
  *   fold_01
      * Imagem_1
      * Imagem_2
      * ...
      * Imagem_*N*

  * fold_02
  * fold_03
  * ...
  * fold_*N*

In [ ]:
# enderço da base de dados divida em folds

base_path = './COPE_UNIFESP_NBfolds'

Estruturas auxiliares

In [ ]:
# dicts utilizados para ajustar a rotina do gradcam para cada modelo

# endereços para gravação dos resultados
path_dict={"VGG16": './VGG16_GradCAM',
           "ResNet50": './ResNet50_GradCAM',
           "SENet50": './SENet50_GradCAM',
           "IncepV3": './IncepV3_GradCAM',
           "NCNN": './NCNN_GradCAM'}

# dimensões das imagens
size_dict={"VGG16": (224,224),
           "ResNet50": (224,224),
           "SENet50": (224,224),
           "IncepV3": (299,299),
           "NCNN": (120,120)}

# Nome da camada analisada em cada modelo
# Trata-se da camada imediatamente anterior a última cada
# de pooling do modelo.
# Este parâmetro muda conforme o arquivo .h5 utilizado, 
# portanto, talvez seja necessário analisar os nomes das camadas
# da CNN antes de prosseguir.
conv_layer_dict = {"VGG16": "conv5_3",
                  "ResNet50": "activation_48",
                  "SENet50": "activation_161",
                  "IncepV3": "mixed10",
                  "NCNN": "conv_2x2_"}

# nome do arquivo .h5 do modelo
model_dict = {"VGG16": "VGG16.h5",
              "ResNet50": "ResNet50.h5",
              "SENet50": "SENet50.h5",
              "IncepV3": "InceptionV3.h5",
              "NCNN": "NCNN.h5"}

Rotina de aplicação do Grad-CAM e gravação dos resultados

In [ ]:
# endereço para gravação dos resultados
file_path= path_dict[modelo]

# dimensões das imagens para o modelo escolhido
img_size= size_dict[modelo]

# nome da camada para aplicação do Grad-CAM
last_conv_layer_name = conv_layer_dict[modelo]

# nome do arquivo .h5 do modelo
model_file = model_dict[modelo]

# O modelo é instaciado duas vezes
# Na primeira, o modelo é carregado normalmente
rede = load_model(model_file)

# Na segunda, a ativação da camada de saída é desabilitada
model = load_model(model_file)
# Remoção da função softmax do modelo
model.layers[-1].activation = None

# lista de resultados
rlist=[]

# varrendo folds da base de dados
for fold in os.listdir(base_path):

  # endereço do fold
  fold_path = osp.join(base_path,fold)

  # varrendo imagens do fold
  for img_name in os.listdir(fold_path):
    
    # endereço da imagem
    img_path = osp.join(fold_path,img_name)

    # verdadeira classe da imagem
    real_label=''

    # imagens "Sem Dor" da base iCOPE possuem a palavra 'rest' no nome
    # imagens "Sem Dor" da base UNIFESP possuem o termo 'sem_dor' no nome
    if ('sem' in img_name) or ('rest' in img_name):
      real_label = 'No_pain'
    else:
      real_label = 'Pain'
    
    # lista de possíveis classes da previsão do modelo
    lab_preds=['Pain','No_pain']
    
    # conversao da imagem em array e pré-processamento
    img_array = preprocess_input(get_img_array(img_path, size=img_size), modelo)

    # classificacao pelo modelo original, isto é, a versão que manteve a ativacao
    # da saída
    preds = rede.predict(img_array)

    # indíce da classe de maior probabilidade na saída softmax
    index = np.argmax(preds)
    
    # determina se o modelo acertou a classificação, ou seja, se a classe prevista
    # é igual a classe verdadeira
    correto = 'Right' if real_label == lab_preds[index] else 'Wrong'
    
    # a confiabilidade do modelo na classificação
    conf = str(round(preds[0][index]*100,2)) +"%"

    # um linha do relatório que terá os resultados de cada classificação
    resultado = (fold,img_name.replace('.png',''),real_label,lab_preds[index],conf,correto)
    rlist.append(resultado)
    
    # aplicação do GradCAM para obtenção do mapa de ativação de classe
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=index)
    
    # salvando o mapa de ativação da pasta geral
    cam_path_all=osp.join(file_path,'All',correto,modelo + "_"+img_name)
    save_gradcam(img_path, heatmap, img_size, cam_path_all, alpha=0.4)

    # salvando o mapa de ativação da pasta exclusiva do fold em questão
    cam_path_fold=osp.join(file_path,fold,correto,modelo + "_"+img_name)
    save_gradcam(img_path, heatmap, img_size, cam_path_fold, alpha=0.4)

# criação do relatório (planilha) com os resultados de cada classificacao
columns = ["Fold","Image","Class","Pred","Conf","Right?"]
df = pd.DataFrame(data = rlist,columns = columns)
csv_name = osp.join(file_path,modelo + ".csv")
df.to_csv(csv_name,index=False,sep=";",decimal=',')